web scrapeing 

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re

# Function to scrape product info
def scrape_pzn_details(pzn):
    try:
        url = f"https://www.shop-apotheke.com/arzneimittel/{pzn}/"  
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            return {"pzn": pzn, "error": f"Failed to retrieve data, status code: {response.status_code}"}

        # Parse the HTML
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract title
        title = soup.find("title").text.strip() if soup.find("title") else "N/A"

        # Extract price (from meta description or similar tag)
        meta_desc = soup.find("meta", attrs={"name": "description"})
        description = meta_desc["content"] if meta_desc else "N/A"
        
        # Extract price from description (or any available price tag)
        price = "N/A"
        if "€" in description:
            price = description.split("€")[1].split(" ")[0].strip()

        # Extract canonical URL (or use a pattern to get the product URL)
        canonical_url = soup.find("link", rel="canonical")
        product_url = canonical_url["href"] if canonical_url else "N/A"

        # Extract category/brand from description (if available)
        category = "N/A"
        brand = "N/A"
        if description:
            # Example for brand/category identification logic, modify as necessary
            if "Eucerin" in description:
                brand = "Eucerin"
            if "Sunscreen" in description:
                category = "Sunscreen"

        return {
            "pzn": pzn,
            "title": title,
            "price": price,
            "description": description
           
            
        }
    except Exception as e:
        return {"pzn": pzn, "error": str(e)}


pzn_data = pd.read_csv(r"C:\Users\NIMISHA\Downloads\20241114_export_pharmacy_receiptdata_products.csv", header=None)
pzn_data.columns = ['pzn', 'title']
pzn_counts = pzn_data['pzn'].value_counts()
selected_pzns = pzn_counts.head(200).tail(150) 
# Scraping data for selected PZNs
results = []
for pzn in selected_pzns.index:
    print(f"Scraping details for PZN: {pzn}")
    result = scrape_pzn_details(pzn)
    results.append(result)
    time.sleep(2)  # Delay to avoid rate limiting

# Create a DataFrame and save the results to CSV
scraped_df = pd.DataFrame(results)
scraped_df.to_csv(r"C:\Users\NIMISHA\Desktop\Assigment6 new\scraped_pzn_data.csv", index=False)

print("Scraping completed and saved to CSV.") 

Scraping details for PZN: 18201503
Scraping details for PZN: 18215149
Scraping details for PZN: 12550409
Scraping details for PZN: 17674903
Scraping details for PZN: 2830585
Scraping details for PZN: 18110232
Scraping details for PZN: 10787337
Scraping details for PZN: 9651609
Scraping details for PZN: 16599832
Scraping details for PZN: 9461168
Scraping details for PZN: 17553447
Scraping details for PZN: 17268534
Scraping details for PZN: 18110226
Scraping details for PZN: 16760115
Scraping details for PZN: 16703229
Scraping details for PZN: 7641311
Scraping details for PZN: 17313387
Scraping details for PZN: 679612
Scraping details for PZN: 16330656
Scraping details for PZN: 16153036
Scraping details for PZN: 15201158
Scraping details for PZN: 2830579
Scraping details for PZN: 1852188
Scraping details for PZN: 1448435
Scraping details for PZN: 2483617
Scraping details for PZN: 16762410
Scraping details for PZN: 14163898
Scraping details for PZN: 4979274
Scraping details for PZN: 16760

Define their category

In [23]:

# Load the scraped data
file_path = r"C:\Users\NIMISHA\Desktop\Assigment6 new\scraped_pzn_data.csv"
scraped_df = pd.read_csv(file_path)

# Step 1: Remove entries where both title and description are missing
scraped_df.dropna(subset=["title", "description"], how="all", inplace=True)

# Step 2: Ensure 'category' column exists
if 'category' not in scraped_df.columns:
    scraped_df['category'] = 'Uncategorized'  # Default to 'Uncategorized' if the column does not exist

# Step 3: Define broader categories with relevant keywords
category_groups = {
    "Sun & Skin Care": ["sun", "sunscreen", "uv", "spf", "photoaging", "skin", "cream", "anti-pigment", "cerave", "eucerin", "hydrocortison", "eczema", "sensitive"],
    "Pain & Wound Care": ["TIGER BALM", "wund", "aspirin", "fußpilzerkrankungen", "pain", "fingerverband", "schmerz", "voltaren", "gel", "diclofenac", "ibu-lysin", "ibuprofen", "wound", "hansaplast", "plaster", "spray", "aqua protect", "sensitive xxl"],
    "Cold, Allergy & Respiratory": ["cold", "allergy", "nasal", "mometahexal", "contramutan", "nasenspray", "hustensaft", "inhaler", "spray"],
    "Oral & Dental Care": ["oral", "tepe", "interdental", "mouth", "dental", "toothpaste"],
    "Baby & Kids": ["baby", "junior", "kids"],
    "Vitamins & Supplements": ["vitamin", "folio", "dekristol", "b5", "supplement", "magnesium", "imodium", "multivitamin", "zinc", "iron"],
    "Insect Repellents & Protection": ["insect", "repellent", "anti brumm", "mosquito"],
    "Cold Sore & Herpes Treatment": ["herpes", "pencivir", "compeed", "acyclovir"],
    "Disinfection & Antiseptics": ["disinfect", "pvp-jod", "octenisept", "antiseptic", "iodine"],
    "Serum & Skincare": ["serum", "hyaluronic", "retinol", "niacinamide"],
    "Beauty & Cosmetics": ["beauty", "cosmetic", "foundation", "lipstick", "makeup", "mascara", "blush"],
    "Creams & Ointments": ["cream", "lotion", "ointment", "moisturizer", "gel"],
    "Tablets & Capsules": ["tabs", "capsule", "pills", "dragees"],
    "Smoking Cessation": ["nicorette", "tx pflaster", "smoking cessation", "nicotine patch"]
}

# Step 4: Compile category-specific regex patterns for faster matching
category_patterns = {}
for category, keywords in category_groups.items():
    # Create a regex pattern that matches any of the keywords in the category
    pattern = r'\b(?:' + '|'.join(map(re.escape, keywords)) + r')\b'
    category_patterns[category] = re.compile(pattern, re.IGNORECASE)

# Step 5: Assign category based on keywords using the compiled regex patterns
def assign_category(row):
    title = str(row["title"]).lower() if pd.notna(row["title"]) else ""
    description = str(row["description"]).lower() if pd.notna(row["description"]) else ""
    combined_text = title + " " + description  # Combine both title and description for better matching

    for category, pattern in category_patterns.items():
        if pattern.search(combined_text):  # Match against the compiled pattern
            return category
    return "Uncategorized"

# Apply the categorization function
scraped_df["category"] = scraped_df.apply(assign_category, axis=1)

# Step 6: Remove unwanted columns ('brand' and 'error') if they exist
columns_to_remove = ['brand', 'error']
scraped_df.drop(columns=[col for col in columns_to_remove if col in scraped_df.columns], inplace=True)

# Step 7: Save categorized data into group-wise CSVs
output_folder = r"C:\Users\NIMISHA\Desktop\Assigment6 new\categorized_data"
os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

# Save products for each category in separate CSV files
for category in category_groups.keys():
    category_df = scraped_df[scraped_df["category"] == category]
    if not category_df.empty:
        # Secure file naming by removing any special characters
        category_file = os.path.join(output_folder, f"{category.replace(' ', '_').lower()}.csv")
        category_df.to_csv(category_file, index=False)
        print(f"Saved {category} products to {category_file}")


Saved Sun & Skin Care products to C:\Users\NIMISHA\Desktop\Assigment6 new\categorized_data\sun_&_skin_care.csv
Saved Pain & Wound Care products to C:\Users\NIMISHA\Desktop\Assigment6 new\categorized_data\pain_&_wound_care.csv
Saved Cold, Allergy & Respiratory products to C:\Users\NIMISHA\Desktop\Assigment6 new\categorized_data\cold,_allergy_&_respiratory.csv
Saved Oral & Dental Care products to C:\Users\NIMISHA\Desktop\Assigment6 new\categorized_data\oral_&_dental_care.csv
Saved Baby & Kids products to C:\Users\NIMISHA\Desktop\Assigment6 new\categorized_data\baby_&_kids.csv
Saved Vitamins & Supplements products to C:\Users\NIMISHA\Desktop\Assigment6 new\categorized_data\vitamins_&_supplements.csv
Saved Insect Repellents & Protection products to C:\Users\NIMISHA\Desktop\Assigment6 new\categorized_data\insect_repellents_&_protection.csv
Saved Cold Sore & Herpes Treatment products to C:\Users\NIMISHA\Desktop\Assigment6 new\categorized_data\cold_sore_&_herpes_treatment.csv
Saved Disinfecti

Bonus task 

In [26]:


def calculate_pzn_check_digit(pzn_base):
    """
    Calculate the check digit for a PZN using the Modulo 11 algorithm.
    Args:
        pzn_base (str): The base part of the PZN (7 digits).
    Returns:
        int: The check digit, or -1 if the calculation results in 10 (invalid PZN).
    """
    if not pzn_base.isdigit() or len(pzn_base) != 7:
        return -1  # Invalid input
    
    weights = range(1, 8)  # Weights from 1 to 7
    weighted_sum = sum(int(digit) * weight for digit, weight in zip(pzn_base, weights))
    check_digit = weighted_sum % 11
    
    return check_digit if check_digit != 10 else -1  # Return -1 for invalid PZNs


def validate_pzn(pzn):
    """
    Validate a complete PZN by checking its check digit.
    Args:
        pzn (str): The full PZN (8 digits, including the check digit).
    Returns:
        bool: True if the PZN is valid, False otherwise.
    """
    if not pzn.isdigit() or len(pzn) != 8:
        return False  # Invalid format or length
    
    pzn_base, actual_check_digit = pzn[:-1], int(pzn[-1])
    expected_check_digit = calculate_pzn_check_digit(pzn_base)
    
    return actual_check_digit == expected_check_digit


# Updated file path
csv_file_path = r"C:\Users\NIMISHA\Downloads\20241114_export_pharmacy_receiptdata_products.csv"

# Load the dataset (assuming no header)
cleaned_data = pd.read_csv(csv_file_path, header=None, dtype=str)

# Assign column names manually (assuming first column is PZN, second is title/product name)
cleaned_data.columns = ['pzn', 'title']

# Ensure 'pzn' is a string, remove spaces, and pad with leading zeros if necessary
cleaned_data['pzn'] = cleaned_data['pzn'].astype(str).str.strip().str.zfill(8)  # Ensures all PZNs are 8 digits

# Apply PZN validation
cleaned_data['is_valid_pzn'] = cleaned_data['pzn'].apply(validate_pzn)

# Extract invalid PZNs
invalid_pzns = cleaned_data[~cleaned_data['is_valid_pzn']]

if not invalid_pzns.empty:
    print(f"Found {len(invalid_pzns)} invalid PZNs:")
    print(invalid_pzns[['pzn', 'title']])

    # Save invalid PZNs to a CSV for review
    invalid_pzns.to_csv(r"C:\Users\NIMISHA\Downloads\invalid_pzns.csv", index=False)
    print("Invalid PZNs saved to 'invalid_pzns.csv'.")
else:
    print("All PZNs are valid.")


All PZNs are valid.


In [28]:


def calculate_pzn_check_digit(pzn_base):
    """
    Calculate the check digit for a PZN using the Modulo 11 algorithm.
    Args:
        pzn_base (str): The base part of the PZN (7 digits).
    Returns:
        int: The check digit, or -1 if the calculation results in 10 (invalid PZN).
    """
    if not pzn_base.isdigit() or len(pzn_base) != 7:
        raise ValueError("PZN base must be a 7-digit numeric string.")
    
    weights = range(1, 8)  # Weights from 1 to 7
    weighted_sum = sum(int(digit) * weight for digit, weight in zip(pzn_base, weights))
    check_digit = weighted_sum % 11

    return check_digit if check_digit != 10 else -1  # Return -1 for invalid PZNs


def validate_pzn(pzn):
    """
    Validate a complete PZN by checking its check digit.
    Args:
        pzn (str): The full PZN (8 digits, including the check digit).
    Returns:
        bool: True if the PZN is valid, False otherwise.
    """
    pzn = str(pzn).strip().zfill(8)  # Ensure PZN is a string and pad missing zeros
    
    if not pzn.isdigit() or len(pzn) != 8:
        return False  # Invalid format or length
    
    pzn_base, actual_check_digit = pzn[:-1], int(pzn[-1])
    expected_check_digit = calculate_pzn_check_digit(pzn_base)
    
    return actual_check_digit == expected_check_digit


#  Load and Process the Data
cleaned_file_path = r"C:\Users\NIMISHA\Downloads\20241114_export_pharmacy_receiptdata_products.csv"

# Load CSV without headers 
cleaned_data = pd.read_csv(cleaned_file_path, dtype=str, header=None)

# Assign column names manually if missing
cleaned_data.columns = ["pzn", "title"]  # Update based on actual data

# Ensure 'pzn' is always a string and preserve leading zeros
cleaned_data["pzn"] = cleaned_data["pzn"].astype(str).str.strip().str.zfill(8)

#  Apply PZN Validation**
cleaned_data['is_valid_pzn'] = cleaned_data['pzn'].apply(validate_pzn)

# Step 3: Extract and Save Invalid PZNs
invalid_pzns = cleaned_data[~cleaned_data['is_valid_pzn']]
print(f"Found {len(invalid_pzns)} invalid PZNs:")
print(invalid_pzns[['pzn', 'title']])

# Save invalid PZNs to a CSV for review
invalid_pzns.to_csv("invalid_pzns.csv", index=False)
print("Invalid PZNs saved to 'invalid_pzns.csv'.")

#  Save the cleaned & validated data
cleaned_data.to_csv("cleaned_validated_pzn_data.csv", index=False)
print("Cleaned and validated PZN data saved to 'cleaned_validated_pzn_data.csv'.")


Found 0 invalid PZNs:
Empty DataFrame
Columns: [pzn, title]
Index: []
Invalid PZNs saved to 'invalid_pzns.csv'.
Cleaned and validated PZN data saved to 'cleaned_validated_pzn_data.csv'.
